In [2]:
!pip install polars langdetect transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=3a1b2d9a3a9c6fcde5462a1bfd125631dd7fa55d895e37b129d55d37bb30a445
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [10]:
!pip install --upgrade polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 32.2 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.21.0
    Uninstalling polars-1.21.0:
      Successfully uninstalled polars-1.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-polars-cu12 25.2.2 requires polars<1.22,>=1.20, but you have polars 1.27.1 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

# For files in a folder named "documents" under /content
base_folder = '/content'
documents_path = os.path.join(base_folder, 'documents')

In [6]:
print("Documents path:", documents_path)
print("Directory exists?", os.path.isdir(documents_path))
!ls -l {documents_path}

Documents path: /content/documents
Directory exists? False
ls: cannot access '/content/documents': No such file or directory


In [7]:
from google.colab import files

uploaded = files.upload()  # This will open a file-selection dialog

# List the files in /content to verify upload:
!ls -l /content

Saving documents_000021.jsonl to documents_000021.jsonl
total 22268
-rw-r--r-- 1 root root 22790426 Apr 14 10:36 documents_000021.jsonl
drwx------ 7 root root     4096 Apr 14 10:34 drive
drwxr-xr-x 1 root root     4096 Apr 10 13:37 sample_data


In [9]:
import polars as pl
import os
import sys

# Adjust the file path to match your uploaded file.
file_path = "/content/documents_000021.jsonl"

if not os.path.exists(file_path):
    sys.exit(f"ERROR: File not found at {file_path}")
else:
    print(f"File found at {file_path}")

# Define an explicit schema.
# (Customize the schema to match the actual structure of your JSONL file.)
schema = {
    "id": pl.Utf8,
    "title": pl.Utf8,
    "abstract": pl.Utf8,
    "authors": pl.List(pl.Struct([pl.Field("name", pl.Utf8)])),
    "createdDate": pl.Utf8,
    "doi": pl.Utf8,
    "arxivId": pl.Utf8,
    "pubmedId": pl.Utf8,
    "magId": pl.Utf8,
    "oaiIds": pl.List(pl.Utf8),
    "links": pl.List(pl.Struct([
        pl.Field("type", pl.Utf8),
        pl.Field("url", pl.Utf8)
    ])),
    "publishedDate": pl.Utf8,
    "updatedDate": pl.Utf8
}

# Read the JSONL file using the explicit schema:
try:
    df = pl.read_ndjson(file_path, schema=schema)
    print(f"DataFrame shape: {df.shape}")
except Exception as e:
    sys.exit(f"Error reading the file: {e}")


File found at /content/documents_000021.jsonl
DataFrame shape: (14265, 13)


In [1]:
# Step 0: Upgrade/install required packages.
!pip install --upgrade polars transformers sentencepiece sacremoses

# Optional: Force CUDA errors to be reported synchronously for debugging
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Import libraries
import os
import re
import time
import sys
import polars as pl
from langdetect import detect, LangDetectException

# ------------------------------
# 1. Set File Path and Define Explicit Schema
# ------------------------------

file_path = "/content/documents_000021.jsonl"
if not os.path.exists(file_path):
    sys.exit(f"ERROR: File not found at {file_path}")
else:
    print(f"File found at {file_path}")

# Define an explicit schema; adjust field names/types to match your JSONL file.
schema = {
    "id": pl.Utf8,
    "title": pl.Utf8,
    "abstract": pl.Utf8,
    "authors": pl.List(pl.Struct([pl.Field("name", pl.Utf8)])),
    "createdDate": pl.Utf8,
    "doi": pl.Utf8,
    "arxivId": pl.Utf8,
    "pubmedId": pl.Utf8,
    "magId": pl.Utf8,
    "oaiIds": pl.List(pl.Utf8),
    "links": pl.List(pl.Struct([
         pl.Field("type", pl.Utf8),
         pl.Field("url", pl.Utf8)
    ])),
    "publishedDate": pl.Utf8,
    "updatedDate": pl.Utf8
}

print("Reading JSONL file with explicit schema...")
try:
    df = pl.read_ndjson(file_path, schema=schema)
    print(f"DataFrame shape: {df.shape}")
except Exception as e:
    sys.exit(f"Error reading JSONL file: {e}")

# ------------------------------
# 2. Define Helper Functions (Text Cleaning & Language Detection)
# ------------------------------

def clean_text_polars(text: str | None) -> str | None:
    if not isinstance(text, str):
        return None
    cleaned = re.sub(r'Comment:.*', '', text, flags=re.IGNORECASE | re.DOTALL)
    cleaned = cleaned.replace('\n', ' ').replace('\u00a0', ' ')
    cleaned = cleaned.strip()
    cleaned = re.sub(r'\s{2,}', ' ', cleaned)
    return cleaned if cleaned else None

def detect_language_polars(text: str | None) -> str:
    if not isinstance(text, str) or len(text.strip()) < 10:
        return "null_or_short"
    try:
        return detect(text)
    except LangDetectException:
        return "undetermined"
    except Exception:
        return "detection_error"

# ------------------------------
# 3. Progress Tracker for Language Detection
# ------------------------------

class LanguageDetectorWithProgress:
    def __init__(self, total_count=None, print_step=1000):
        self.counter = 0
        self.total_count = total_count
        self.print_step = print_step

    def detect_and_count(self, text: str | None) -> str:
        lang = detect_language_polars(text)
        self.counter += 1
        if self.counter % self.print_step == 0:
            if self.total_count is not None:
                print(f"[{time.strftime('%H:%M:%S')}] Processed {self.counter}/{self.total_count} abstracts: detected language -> {lang}")
            else:
                print(f"[{time.strftime('%H:%M:%S')}] Processed {self.counter} abstracts: detected language -> {lang}")
        return lang

    def reset(self):
        self.counter = 0

    def get_count(self):
        return self.counter

# ------------------------------
# 4. Clean Abstracts & Detect Language
# ------------------------------

print("Cleaning abstract texts...")
df = df.with_columns(
    pl.col("abstract")
      .map_elements(clean_text_polars, return_dtype=pl.Utf8, strategy="threading")
      .alias("abstract_cleaned")
)

print("Starting language detection...")
total_valid = df.filter(pl.col("abstract_cleaned").is_not_null()).height
print(f"Total valid abstracts for language detection: {total_valid}")

lang_tracker = LanguageDetectorWithProgress(total_count=total_valid, print_step=1000)
df = df.with_columns(
    pl.col("abstract_cleaned")
      .map_elements(lang_tracker.detect_and_count, return_dtype=pl.Utf8)
      .alias("detected_language")
)
print("Language detection complete.")
print(f"Total abstracts processed for language detection: {lang_tracker.get_count()}")

print("\nSample rows (abstract_cleaned and detected_language):")
print(df.select(["abstract_cleaned", "detected_language"]).head(10))

# ------------------------------
# 5. Display Language Distribution & Filter Non-English Abstracts
# ------------------------------

print("\nLanguage distribution (using Series.value_counts()):")
language_counts = df["detected_language"].value_counts()
print(language_counts)

print("\nFiltering non-English abstracts (excluding 'en', 'null_or_short', and 'undetermined'):")
non_english_df = df.filter(~pl.col("detected_language").is_in(["en", "null_or_short", "undetermined"]))
print(f"Non-English abstracts count: {non_english_df.height}")
print(non_english_df.select(["abstract_cleaned", "detected_language"]).head(10))

# ------------------------------
# 6. Translate Non-English Abstracts (Using Transformers Pipeline)
# ------------------------------

from transformers import pipeline
import torch

# Set device to GPU if available; else CPU.
device = 0 if torch.cuda.is_available() else -1
try:
    translator = pipeline(
        "translation",
        model="Helsinki-NLP/opus-mt-mul-en",
        device=device
    )
    print(f"Translator loaded on device: {'GPU' if device == 0 else 'CPU'}")
except Exception as e:
    print(f"Error initializing translator on device {device}: {e}")
    print("Falling back to CPU.")
    device = -1
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en", device=device)
    print("Translator loaded on CPU.")

def batch_translate(texts, batch_size=32):
    translated_texts = []
    total = len(texts)
    print(f"Translating {total} non-English abstracts...")
    for i in range(0, total, batch_size):
        batch = texts[i : i + batch_size]
        try:
            translations = translator(
                batch,
                truncation=True,
                max_length=512,
                padding=True
            )
        except Exception as e:
            print(f"Error with batch starting at index {i}: {e}")
            print("Falling back to CPU translation for this batch.")
            cpu_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en", device=-1)
            translations = cpu_translator(
                batch,
                truncation=True,
                max_length=512,
                padding=True
            )
        translated_texts.extend([t["translation_text"] for t in translations])
        print(f"Processed {min(i + batch_size, total)}/{total} abstracts in this batch")
    return translated_texts

# Extract non-English abstracts as a list.
non_english_texts = non_english_df.select("abstract_cleaned").to_series().to_list()

# Perform batch translation.
translated_abstracts = batch_translate(non_english_texts, batch_size=32)

print("\nSample translation outputs:")
for original, translation in zip(non_english_texts[:5], translated_abstracts[:5]):
    print("Original:", original)
    print("Translated:", translation)
    print("----")


File found at /content/documents_000021.jsonl
Reading JSONL file with explicit schema...
DataFrame shape: (14265, 13)
Cleaning abstract texts...
Starting language detection...
Total valid abstracts for language detection: 11648
[10:55:30] Processed 1000/11648 abstracts: detected language -> pt
[10:55:37] Processed 2000/11648 abstracts: detected language -> en
[10:55:42] Processed 3000/11648 abstracts: detected language -> en
[10:55:46] Processed 4000/11648 abstracts: detected language -> en
[10:55:50] Processed 5000/11648 abstracts: detected language -> en
[10:55:54] Processed 6000/11648 abstracts: detected language -> en
[10:55:59] Processed 7000/11648 abstracts: detected language -> en
[10:56:02] Processed 8000/11648 abstracts: detected language -> en
[10:56:07] Processed 9000/11648 abstracts: detected language -> en
[10:56:11] Processed 10000/11648 abstracts: detected language -> id
[10:56:15] Processed 11000/11648 abstracts: detected language -> en
Language detection complete.
Tota

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


Translator loaded on device: GPU
Translating 1892 non-English abstracts...
Error with batch starting at index 0: The following `model_kwargs` are not used by the model: ['padding'] (note: typos in the generate arguments will also show up in this list)
Falling back to CPU translation for this batch.


Device set to use cpu


ValueError: The following `model_kwargs` are not used by the model: ['padding'] (note: typos in the generate arguments will also show up in this list)

In [2]:
# Upgrade Polars and install required packages for translation
!pip install --upgrade polars
!pip install transformers sentencepiece torch accelerate sacremoses # Added torch, accelerate, sacremoses

import os
# --- Debugging Addition ---
# Set CUDA_LAUNCH_BLOCKING=1 to get more specific error messages from CUDA.
# This makes GPU operations synchronous and slows things down,
# so remove or comment it out for production runs if errors are resolved.
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # Keep commented out unless debugging CUDA issues
# --- End Debugging Addition ---

import re
import time
import sys
import polars as pl
from langdetect import detect, LangDetectException
from transformers import pipeline
import torch

# ------------------------------
# 1. Set the File Path and Define an Explicit Schema
# ------------------------------

file_path = "/content/documents_000021.jsonl" # Make sure this path is correct for your environment
if not os.path.exists(file_path):
    sys.exit(f"ERROR: File not found at {file_path}")
else:
    print(f"File found at {file_path}")

# Define an explicit schema matching your JSONL file structure.
schema = {
    "id": pl.Utf8,
    "title": pl.Utf8,
    "abstract": pl.Utf8,
    "authors": pl.List(pl.Struct([pl.Field("name", pl.Utf8)])),
    "createdDate": pl.Utf8,
    "doi": pl.Utf8,
    "arxivId": pl.Utf8,
    "pubmedId": pl.Utf8,
    "magId": pl.Utf8,
    "oaiIds": pl.List(pl.Utf8),
    "links": pl.List(pl.Struct([
         pl.Field("type", pl.Utf8),
         pl.Field("url", pl.Utf8)
    ])),
    "publishedDate": pl.Utf8,
    "updatedDate": pl.Utf8
}

print("Reading JSONL file with explicit schema...")
try:
    # Consider adding `ignore_errors=True` if some lines might be malformed,
    # though this might hide other issues.
    df = pl.read_ndjson(file_path, schema=schema)
    print(f"DataFrame shape: {df.shape}")
except Exception as e:
    sys.exit(f"Error reading JSONL file: {e}")

# ------------------------------
# 2. Define Helper Functions
# ------------------------------

def clean_text_polars(text: str | None) -> str | None:
    if not isinstance(text, str):
        return None
    # Remove "Comment: ..." sections common in arXiv abstracts
    cleaned = re.sub(r'Comment:.*', '', text, flags=re.IGNORECASE | re.DOTALL)
    # Replace newline characters and non-breaking spaces with a single space
    cleaned = cleaned.replace('\n', ' ').replace('\u00a0', ' ')
    # Remove leading/trailing whitespace
    cleaned = cleaned.strip()
    # Replace multiple spaces with a single space
    cleaned = re.sub(r'\s{2,}', ' ', cleaned)
    return cleaned if cleaned else None # Return None if the string becomes empty

def detect_language_polars(text: str | None) -> str:
    # Handle None, non-string inputs, or very short strings
    if not isinstance(text, str) or len(text.strip()) < 10: # Min length for reliable detection
        return "null_or_short"
    try:
        # Use langdetect library
        return detect(text)
    except LangDetectException:
        # Langdetect couldn't determine the language
        return "undetermined"
    except Exception as e:
        # Catch any other unexpected errors during detection
        print(f"Unexpected error in language detection for text: '{text[:50]}...': {e}")
        return "detection_error"

# ------------------------------
# 3. Progress Tracker for Language Detection
# ------------------------------

class LanguageDetectorWithProgress:
    def __init__(self, total_count=None, print_step=1000):
        self.counter = 0
        self.total_count = total_count
        self.print_step = print_step
        self.start_time = time.time()

    def detect_and_count(self, text: str | None) -> str:
        lang = detect_language_polars(text)
        self.counter += 1
        if self.counter % self.print_step == 0 or self.counter == self.total_count:
            elapsed_time = time.time() - self.start_time
            rate = self.counter / elapsed_time if elapsed_time > 0 else 0
            if self.total_count is not None:
                 print(f"[{time.strftime('%H:%M:%S')}] Processed {self.counter}/{self.total_count} abstracts "
                       f"(~{rate:.1f}/sec). Last detected: {lang}")
            else:
                 print(f"[{time.strftime('%H:%M:%S')}] Processed {self.counter} abstracts "
                       f"(~{rate:.1f}/sec). Last detected: {lang}")
        return lang

    def reset(self):
        self.counter = 0
        self.start_time = time.time()

    def get_count(self):
        return self.counter

# ------------------------------
# 4. Clean the Abstracts & Detect Language
# ------------------------------

print("\nCleaning abstract texts...")
# Use map_elements for applying the Python function row-wise
# Using 'thread_local' strategy for potential speedup while aiming for stability
df = df.with_columns(
    pl.col("abstract")
      .map_elements(clean_text_polars, return_dtype=pl.Utf8, strategy="thread_local")
      .alias("abstract_cleaned")
)

print("\nStarting language detection...")
# Filter out rows where cleaning resulted in None before counting
total_valid = df.filter(pl.col("abstract_cleaned").is_not_null()).height
print(f"Total valid abstracts for language detection: {total_valid}")

# Initialize the progress tracker
lang_tracker = LanguageDetectorWithProgress(total_count=total_valid, print_step=1000)

# Apply language detection using the tracker
df = df.with_columns(
    pl.col("abstract_cleaned")
      .map_elements(lang_tracker.detect_and_count, return_dtype=pl.Utf8, strategy="thread_local")
      .alias("detected_language")
)
print("Language detection complete.")
print(f"Total abstracts processed for language detection: {lang_tracker.get_count()}")

print("\nSample rows (abstract_cleaned and detected_language):")
print(df.select(["abstract_cleaned", "detected_language"]).head(10))

# ------------------------------
# 5. Display Language Distribution and Filter Non-English Abstracts
# ------------------------------

print("\nLanguage distribution (using Series.value_counts()):")
# Calculate and print value counts for the detected languages
language_counts = df.get_column("detected_language").value_counts()
print(language_counts)

print("\nFiltering non-English abstracts...")
# Filter the DataFrame to keep only rows where detected_language is not 'en'
# Also explicitly exclude null/short/error categories
non_english_df = df.filter(
    (pl.col("detected_language") != "en") &
    (pl.col("detected_language") != "null_or_short") &
    (pl.col("detected_language") != "undetermined") &
    (pl.col("detected_language") != "detection_error") &
    (pl.col("abstract_cleaned").is_not_null()) # Ensure abstract is not null
)
print(f"Found {non_english_df.height} non-English abstracts for translation.")
print("\nSample non-English abstracts:")
print(non_english_df.select(["abstract_cleaned", "detected_language"]).head(10))

# ------------------------------
# 6. Translate Non-English Abstracts (Using Transformers Pipeline)
# ------------------------------

# --- Diagnostic Step ---
# Check GPU memory right before attempting to load the model
print("\n--- Checking GPU Status (nvidia-smi) ---")
!nvidia-smi
print("----------------------------------------\n")
# --- End Diagnostic Step ---

translator = None # Initialize translator to None
translation_device_used = "None"

try:
    # Determine device: Use GPU if available, otherwise CPU
    if torch.cuda.is_available():
        device = 0 # Use the first available GPU
        print("CUDA (GPU) is available. Attempting to load model on GPU.")
    else:
        device = -1 # Use CPU
        print("CUDA (GPU) not available. Loading model on CPU.")

    # Load the translation pipeline
    # opus-mt-mul-en translates multiple languages ('mul') to English ('en')
    translator = pipeline(
        "translation",
        model="Helsinki-NLP/opus-mt-mul-en",
        device=device,
        torch_dtype=torch.float16 if device == 0 else None # Use float16 on GPU for potential memory saving
    )
    translation_device_used = "GPU" if device == 0 else "CPU"
    print(f"Translator loaded successfully on: {translation_device_used}")

except RuntimeError as e:
    # --- Catch CUDA Errors Specifically ---
    if "CUDA" in str(e):
        print("\n--- ERROR: CUDA Runtime Error during pipeline initialization! ---")
        print(f"Error details: {e}")
        print("This often means insufficient GPU memory (VRAM).")
        print("Suggestions:")
        print("  1. Restart your runtime/kernel to free GPU memory and try again.")
        print("  2. Check nvidia-smi output above for memory usage.")
        print("  3. If memory is low, consider a smaller model or using the CPU.")
        print("Attempting to load the model on CPU as a fallback...")

        try:
            device = -1 # Force CPU
            translator = pipeline(
                "translation",
                model="Helsinki-NLP/opus-mt-mul-en",
                device=device
            )
            translation_device_used = "CPU (Fallback)"
            print(f"Translator loaded successfully on: {translation_device_used}")
        except Exception as cpu_e:
            print(f"\n--- FATAL ERROR: Failed to load model even on CPU! ---")
            print(f"CPU loading error: {cpu_e}")
            print("There might be an issue with the model, downloaded files, or libraries.")
            print("Exiting translation step.")
            # Set non_english_df to empty to skip translation loop
            non_english_df = non_english_df.clear()
    else:
        # Handle other potential RuntimeErrors
        print(f"\n--- ERROR: A non-CUDA RuntimeError occurred during pipeline initialization! ---")
        print(f"Error details: {e}")
        print("Exiting translation step.")
        non_english_df = non_english_df.clear()

except Exception as e:
    # Catch any other exceptions during pipeline loading
    print(f"\n--- FATAL ERROR: An unexpected error occurred during pipeline initialization! ---")
    print(f"Error details: {e}")
    print("Exiting translation step.")
    non_english_df = non_english_df.clear()


# Define the translation function (only if translator loaded successfully)
def batch_translate(texts, batch_size=16): # Reduced default batch size for potentially lower memory usage
    if translator is None:
        print("Translator not available. Skipping translation.")
        return ["Translation skipped due to error"] * len(texts)

    translated_texts = []
    total = len(texts)
    print(f"\nTranslating {total} non-English abstracts using {translation_device_used}...")
    start_time = time.time()

    for i in range(0, total, batch_size):
        batch = texts[i : i + batch_size]
        batch_start_time = time.time()
        try:
            # Perform translation
            # *** CORRECTED LINE: Removed padding=True ***
            translations = translator(batch, truncation=True, max_length=512)
            translated_texts.extend([t["translation_text"] for t in translations])

            batch_time = time.time() - batch_start_time
            processed_count = min(i + batch_size, total)
            print(f"  Processed batch {i//batch_size + 1}/{ (total + batch_size - 1)//batch_size } "
                  f"({processed_count}/{total} abstracts) in {batch_time:.2f}s")

        except Exception as e:
            print(f"--- Error translating batch starting at index {i}: {e} ---")
            # Add placeholders for the failed batch
            translated_texts.extend([f"Error: {e}"] * len(batch))
            # If errors persist, you might consider adding a more robust error
            # handling or retry mechanism here, possibly falling back to CPU
            # for specific problematic batches if the initial load was on GPU.

    total_time = time.time() - start_time
    print(f"Translation finished in {total_time:.2f} seconds.")
    return translated_texts

# Proceed with translation only if there are texts to translate and the translator loaded
translated_abstracts = []
if non_english_df.height > 0 and translator is not None:
    # Extract the texts to be translated
    non_english_texts = non_english_df.get_column("abstract_cleaned").to_list()

    # Perform translation
    translated_abstracts = batch_translate(non_english_texts, batch_size=16) # Use the defined batch size

    # --- Verification ---
    print(f"\nNumber of original non-English texts: {len(non_english_texts)}")
    print(f"Number of translated abstracts received: {len(translated_abstracts)}")

    if len(non_english_texts) == len(translated_abstracts):
        # Create a new DataFrame with original, language, and translation
        # Ensure the lists used here maintain the original order from non_english_df
        translation_results_df = pl.DataFrame({
            "original_abstract": non_english_texts,
            "detected_language": non_english_df.get_column("detected_language").to_list(),
            "translated_abstract": translated_abstracts
        })

        print("\nSample translation outputs:")
        print(translation_results_df.head(10))

        # --- Optional: Merge translations back into the main DataFrame ---
        # If you need the translations in the original df, you would typically:
        # 1. Add a unique temporary ID to non_english_df before extracting text.
        # 2. Add this ID to translation_results_df.
        # 3. Perform a left join from the original `df` onto `translation_results_df` using the ID.
        # Example (conceptual):
        # df_with_id = df.with_row_index("temp_id")
        # non_english_df_with_id = df_with_id.filter(...) # Apply same filters as before
        # non_english_texts = non_english_df_with_id["abstract_cleaned"].to_list()
        # temp_ids = non_english_df_with_id["temp_id"].to_list()
        # ... perform translation ...
        # translation_results_df = pl.DataFrame({
        #    "temp_id": temp_ids,
        #    "translated_abstract": translated_abstracts
        # })
        # df_final = df_with_id.join(translation_results_df, on="temp_id", how="left")
        # df_final = df_final.drop("temp_id") # Clean up

    else:
        print("\n--- WARNING: Mismatch between number of original texts and translations! ---")
        print("Skipping creation of results DataFrame and merging.")

elif translator is None:
    print("\nSkipping translation execution because the translator failed to load.")
else:
    print("\nNo non-English abstracts found to translate.")

print("\nScript finished.")

File found at /content/documents_000021.jsonl
Reading JSONL file with explicit schema...
DataFrame shape: (14265, 13)

Cleaning abstract texts...

Starting language detection...
Total valid abstracts for language detection: 11648
[11:02:30] Processed 1000/11648 abstracts (~202.9/sec). Last detected: en
[11:02:34] Processed 2000/11648 abstracts (~221.7/sec). Last detected: en
[11:02:38] Processed 3000/11648 abstracts (~234.5/sec). Last detected: en
[11:02:42] Processed 4000/11648 abstracts (~226.5/sec). Last detected: en
[11:02:46] Processed 5000/11648 abstracts (~231.3/sec). Last detected: en
[11:02:50] Processed 6000/11648 abstracts (~234.1/sec). Last detected: en
[11:02:56] Processed 7000/11648 abstracts (~225.2/sec). Last detected: en
[11:03:00] Processed 8000/11648 abstracts (~228.5/sec). Last detected: en
[11:03:04] Processed 9000/11648 abstracts (~231.9/sec). Last detected: en
[11:03:09] Processed 10000/11648 abstracts (~226.7/sec). Last detected: en
[11:03:13] Processed 11000/11

Device set to use cuda:0


Translator loaded successfully on: GPU

Translating 1888 non-English abstracts using GPU...


Your input_length: 503 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You

  Processed batch 1/118 (16/1888 abstracts) in 34.52s


...

  Processed batch 22/118 (352/1888 abstracts) in 33.86s


Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You

  Processed batch 23/118 (368/1888 abstracts) in 25.38s


Your input_length: 499 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 512 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


KeyboardInterrupt: 